In [2]:
import os
os.chdir(os.path.expanduser("/home/users/gmenon/workspace/songsLyricsGenerator/src"))

In [3]:
from constants.mir_constants import WAV2VEC2_ARGS
from training.wav2vec2_finetune import Wav2Vec2SpeechRecognition

In [7]:
wav2vec2 = Wav2Vec2SpeechRecognition(wav2vec2_args=WAV2VEC2_ARGS)

/tmp/ipykernel_2835697/1797219780.py:1: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  wav2vec2 = Wav2Vec2SpeechRecognition(WAV2VEC2_ARGS)


In [11]:
len(wav2vec2)

TypeError: 'dict' object cannot be interpreted as an integer

In [ ]:
wav2vec2 = Wav2Vec2SpeechRecognition(WAV2VEC2_ARGS)

print("Length of training and validation datasets" + str(len(wav2vec2))

wav2vec2.model = 
wav2vec2_model = SpeechRecognition(backbone=wav2vec2_args.MODEL_BACKBONE)

# Create the trainer, finetune and save the model
wav2vec2_trainer = flash.Trainer(accumulate_grad_batches=10,
                        precision=16,
                        max_epochs=wav2vec2_args.NUM_EPOCHS, 
                        gpus=wav2vec2_args.NUM_GPUS)

wav2vec2_trainer.finetune(wav2vec2_model,
                 datamodule=datamodule, 
                 strategy=wav2vec2_args.FINETUNE_STRATEGY)

wav2vec2_trainer.save_checkpoint(wav2vec2_args.MODEL_SAVE_PATH)


# Whisper 
whisper_model = SpeechRecognition(backbone=whisper_args.MODEL_BACKBONE)

# Create the trainer, finetune and save the model
whisper_trainer = flash.Trainer(accumulate_grad_batches=10,
                                precision=16, 
                                max_epochs=whisper_args.NUM_EPOCHS,
                                gpus=whisper_args.NUM_GPUS)

whisper_trainer.finetune(whisper_model, datamodule=datamodule, 
                         strategy=whisper_args.FINETUNE_STRATEGY)

whisper_trainer.save_checkpoint(whisper_args.MODEL_SAVE_PATH)


# 4. Predict on audio files!
test_datamodule = SpeechRecognitionData.from_files(
    predict_files=["/home/users/gmenon/workspace/songsLyricsGenerator/test_clip.wav"], 
    batch_size=training_args.BATCH_SIZE)

wav2vec2_predictions = wav2vec2_trainer.predict(wav2vec2_model, 
                                                datamodule=test_datamodule)

print("Wav2vec2 predictions ")
print(wav2vec2_predictions)

print("Whisper predictions ")
whisper_predictions = whisper_trainer.predict(whisper_model, datamodule=test_datamodule)
print(whisper_predictions)
